In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
varns = sorted(['B.1', 'Omicron BA.4/5', 'BA.2.12.1', 'Delta', 'Beta', 'Alpha', 'BA.1'])

In [14]:
# Read in our escape data
date = '2025-06-26'
sb = .6

escape_results = torch.load(f'model_covdb/results.{date}.sb{sb}.pt')
escape_traces = escape_results['traces']
escape_data = escape_results['data']

pool_agnostic_effects = pd.DataFrame({'Mutation':escape_data['features'], 'Immune-Agnostic Effect':escape_traces['β'].mean(dim=0)})



pool_specific_effects = pd.DataFrame(columns=escape_data['features'], index=escape_data['pools_map'], data=escape_traces['β_ω'].mean(dim=0)
                                    ).reset_index(
                                    ).melt(id_vars='index', value_vars=None, var_name='Mutation', value_name='Immune-Specific Effect'
                                    ).rename(columns={'index':'Pool'})
# Subset ranges to where we have ample data on each mutation
min_n = 1  # mutation must have been seen at least once
subset = []
for _, row in pool_specific_effects.iterrows():
    pool_idx = (escape_data['pools_map']==row['Pool']).nonzero()[0][0]
    feat_idx = (np.array(escape_data['features'])==row['Mutation']).nonzero()[0][0]
    num_obs = escape_data['X'][escape_data['pools']==pool_idx, feat_idx].sum().item()
    row = dict(row)
    row['N'] = int(num_obs)
    if num_obs >= min_n:
        subset.append(row)
subset = pd.DataFrame(subset)
pool_specific_effects = pd.merge(pool_specific_effects, subset[['Mutation', 'Pool']], on=['Mutation', 'Pool'], how='inner')
pool_specific_effects_piv = pool_specific_effects.pivot(columns='Pool', 
                                                        index='Mutation', 
                                                        values='Immune-Specific Effect'
                                               ).rename(columns={c: f'{c}-elicited escape' for c in varns})



In [16]:
pd.merge(pool_agnostic_effects, pool_specific_effects_piv.reset_index()).to_csv('supplemental_files/S1.csv', index=False)